In [2]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 
np.random.seed(1337)
df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    # print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all artsits with that movie
    sim_scores = list(enumerate(sim[idx]))
    if not isinstance(sim_scores[0][1],np.ndarray):
        # Sort the artists based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

        # Get the scores of the 10 most similar artsits
        sim_scores = sim_scores[1:21]

        # Get the movie indices
        indices = [i[0] for i in sim_scores]
        scores = [round(i[1]*100, 2) for i in sim_scores]

        # Return the top 20 most similar artsits
        top_10=df3['name'].iloc[indices]
        spot_top10=[]
        spot_top10_names=[]
        for item in top_10:
            spot_top10_names.append(item)
            spot_top10.append(dict_names[str(item)])

        artist=dict_names[str(df3['name'].iloc[idx])]
        
        wantedDict = {
            'mainId': artist,
            'top10': spot_top10,
            'top10Names': spot_top10_names,
            'scores': scores
        }
        return wantedDict
    return {
        'top10': ['wrong format'],
        'top10Names': ['wrong format'],
    }




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [3]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [4]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [5]:
spot1 = pd.read_csv('spotify_results1.csv')
spot2 = pd.read_csv('spotify_results2.csv')
spot3 = pd.read_csv('spotify_results3.csv')
spot1Name = pd.read_csv('spotify_results1_name.csv')
spot2Name = pd.read_csv('spotify_results2_name.csv')
spot3Name = pd.read_csv('spotify_results3_name.csv')
spotDict = {}
spotDictNames = {}

i = 0
for index in spot1:
    if i != 0:
        spotDict[str(index)] = list(spot1[index].to_numpy())
        spotDictNames[str(index)] = list(spot1Name[index].to_numpy())
    i+=1

i = 0
for index in spot2:
    if i != 0:
        spotDict[str(index)] = list(spot2[index].to_numpy())
        spotDictNames[str(index)] = list(spot2Name[index].to_numpy())
    i+=1

i = 0
for index in spot3:
    if i != 0:
        spotDict[str(index)] = list(spot3[index].to_numpy())
        spotDictNames[str(index)] = list(spot3Name[index].to_numpy())
    i+=1

matrix=CVectorizer()
sim,reverse = EuDSim(matrix)

noCutEuDSim = []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (float(count)/20.0)
    noCutEuDSim.append(percentage)

print(noCutEuDSim)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.1, 0.05, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.15, 0.0, 0.0, 0.2, 0.25, 0.05, 0.1, 0.15, 0.15, 0.0, 0.0, 0.0, 0.1, 0.05, 0.05, 0.15, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.0, 0.05, 0.0, 0.25, 0.25, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.05, 0.05, 0.05, 0.0, 0.05, 0.1, 0.0, 0.25, 0.1, 0.1, 0.15, 0.25, 0.0, 0.0, 0.15, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.1, 0.05, 0.15, 0.05, 0.0, 0.05, 0.0, 0.0, 0.25, 0.0, 0.0, 0.2, 0.1, 0.0, 0.25, 0.0, 0.0, 0.2, 0.05, 0.05, 0.0, 0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.0, 0.15, 0.15, 0.05, 0.25, 0.25, 0.05, 0.1, 0.05, 0.2, 0.15, 0.05, 0.15, 0.15, 0.2, 0.0, 0.0, 0.15, 0.1, 0.0, 0.05, 0.05, 0.2, 0.15, 0.0, 0.0, 0.0, 0.0, 0.05, 0.15, 0.05, 0.05, 0.0, 0.0, 0.0, 0.

In [6]:
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)

cutEuDSimCv= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    nameList = spotDictNames[ind]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    popped = 0
    for nameCheck in nameList:
        if nameCheck not in allowedNames:
            i = nameList.index(nameCheck)
            if len(ids) > 0:
                ids.pop(i-popped)
                popped+=1
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
        percentage = 0
        if len(ids) != 0: 
            percentage = round((count/len(ids)), 4)
    cutEuDSimCv.append(percentage)

print(cutEuDSimCv)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0909, 0.0909, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.1667, 0.2, 0.0, 0.0, 0.25, 0.1667, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.125, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2222, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1667, 0.2857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2857, 0.0, 0.0, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1429, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.2, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.3333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333, 0.3, 0.3333, 0.125, 0.25, 0.3333, 0.0, 0.1429, 0.25, 0.0, 0.3333, 0.3333, 0.0, 0.1667, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [7]:
matrix=TfidVectorizer()
sim,reverse = EuDSim(matrix)

noCutEuDTfSim = []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutEuDTfSim.append(percentage)

print(noCutEuDTfSim)

[0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.25, 0.0, 0.05, 0.15, 0.1, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.05, 0.0, 0.0, 0.2, 0.25, 0.0, 0.2, 0.15, 0.2, 0.05, 0.0, 0.0, 0.1, 0.05, 0.05, 0.25, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.15, 0.2, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.15, 0.05, 0.1, 0.05, 0.05, 0.0, 0.05, 0.0, 0.2, 0.0, 0.05, 0.0, 0.2, 0.15, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.0, 0.1, 0.05, 0.1, 0.15, 0.0, 0.0, 0.0, 0.05, 0.2, 0.05, 0.0, 0.0, 0.1, 0.0, 0.1, 0.2, 0.0, 0.0, 0.05, 0.0, 0.05, 0.15, 0.0, 0.2, 0.05, 0.1, 0.15, 0.15, 0.1, 0.0, 0.15, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.1, 0.05, 0.1, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.05, 0.2, 0.1, 0.0, 0.25, 0.05, 0.0, 0.2, 0.05, 0.1, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.15, 0.15, 0.05, 0.2, 0.45, 0.0, 0.0, 0.1, 0.25, 0.2, 0.0, 0.15, 0.2, 0.3, 0.0, 0.1, 0.1, 0.2, 0.0, 0.05, 0.05, 0.2, 0.15, 0.05, 0.1, 0.0, 0.0, 0.05, 0.15, 0.1, 0.1, 0.0, 0.05, 0.0, 0

In [18]:
matrix=TfidVectorizer()
sim,reverse = EuDSim(matrix)

cutEuDSimTf= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    nameList = spotDictNames[ind]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    popped = 0
    for nameCheck in nameList:
        if nameCheck not in allowedNames:
            i = nameList.index(nameCheck)
            if len(ids) > 0:
                ids.pop(i-popped)
                popped+=1
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
        percentage = 0
        if len(ids) != 0: 
            percentage = round((count/len(ids)), 4)
    cutEuDSimTf.append(percentage)


noZeroes = []
for score in cutEuDSimTf:
    if score > 0:
        noZeroes.append(score)
print(np.sum(noZeroes)/len(noZeroes))

0.0


TypeError: object of type 'float' has no len()

In [9]:
matrix=CVectorizer()
im,reverse = CosSim(matrix)

noCutCosSimCV= []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutCosSimCV.append(percentage)

print(noCutCosSimCV)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [10]:
matrix=CVectorizer()
im,reverse = CosSim(matrix)

cutCosSimCv= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    nameList = spotDictNames[ind]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    popped = 0
    for nameCheck in nameList:
        if nameCheck not in allowedNames:
            i = nameList.index(nameCheck)
            if len(ids) > 0:
                ids.pop(i-popped)
                popped+=1
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
        percentage = 0
        if len(ids) != 0: 
            percentage = round((count/len(ids)), 4)
    cutCosSimCv.append(percentage)

print(cutCosSimCv)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [11]:
matrix=TfidVectorizer()
im,reverse = CosSim(matrix)

noCutCosSimTf= []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutCosSimTf.append(percentage)

print(noCutCosSimTf)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [12]:
matrix=TfidVectorizer()
im,reverse = CosSim(matrix)

cutCosSimTf= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    nameList = spotDictNames[ind]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    popped = 0
    for nameCheck in nameList:
        if nameCheck not in allowedNames:
            i = nameList.index(nameCheck)
            if len(ids) > 0:
                ids.pop(i-popped)
                popped+=1
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
        percentage = 0
        if len(ids) != 0: 
            percentage = round((count/len(ids)), 4)
    cutCosSimTf.append(percentage)

print(cutCosSimTf)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [13]:
def Random_tester():
    rand= np.random.randint(0,len(df3),299)
    return rand

sample=Random_tester()
df_sample=pd.DataFrame(sample)
df_sample = df_sample.drop_duplicates()
# df_sample.to_csv("random.csv",index=False)

In [14]:
import requests
from time import sleep

apiKey = 'BQA9NeCisLRR1GoN4NC-CnUxZvjWuFewAgS8ZRFFL35UkarFa-WMFyIqqJIAaIejWDI_Ok_LRdv_qjqw2pF4CqAsTj9fgsAAL3r7er7AGxtGnWpFZ16876PL9SolaTqRru7-gD17Ge9GOzLqm-aTXn7iBalFhabtCJemMKBf99gnD94HCsQgO7QhWiE2L7g'
BASE_URL = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {token}'.format(token=apiKey)
}


rand_df=pd.read_csv("random.csv",index_col=None)
# print(rand_df.head())
spotifyResultsDf1 = pd.DataFrame()
spotifyResultsDf2 = pd.DataFrame()
spotifyResultsDf3 = pd.DataFrame()
spotifyResultsDf1Name = pd.DataFrame()
spotifyResultsDf2Name = pd.DataFrame()
spotifyResultsDf3Name = pd.DataFrame()
i=0
for item,row in rand_df.iterrows():
    id=str(df3['spotifyid'].iloc[row["Num"]])
    topTwentySpotArray = []
    topTwentySpotArrayName = []
    r = requests.get(BASE_URL + 'artists/' + id + '/related-artists', headers=headers)
    rData = r.json()
    for entry in rData['artists']:
        spotId =  entry['id']
        spotName =  entry['name']
        topTwentySpotArray.append(spotId)
        topTwentySpotArrayName.append(spotName)
        
    if(len(topTwentySpotArray) < 20):
        while len(topTwentySpotArray) < 20:
            topTwentySpotArray.append("")
            topTwentySpotArrayName.append("")

    print(i)
    i+=1
    numberWanted = str(row['Num'])
    if i < 100:
        spotifyResultsDf1[numberWanted] = topTwentySpotArray
        spotifyResultsDf1Name[numberWanted] = topTwentySpotArrayName
    elif i >= 100 and i < 200:
        spotifyResultsDf2[numberWanted] = topTwentySpotArray
        spotifyResultsDf2Name[numberWanted] = topTwentySpotArrayName
    else:
        spotifyResultsDf3[numberWanted] = topTwentySpotArray
        spotifyResultsDf3Name[numberWanted] = topTwentySpotArrayName


    sleep(0.4)


spotifyResultsDf1.to_csv('spotify_results1.csv')
spotifyResultsDf2.to_csv('spotify_results2.csv')
spotifyResultsDf3.to_csv('spotify_results3.csv')
spotifyResultsDf1Name.to_csv('spotify_results1_name.csv')
spotifyResultsDf2Name.to_csv('spotify_results2_name.csv')
spotifyResultsDf3Name.to_csv('spotify_results3_name.csv')



KeyError: 'artists'